In [ ]:
import numpy as np
import itertools

1.1. Реализовать функцию REF(), приводящую матрицу к
ступенчатому виду.

In [ ]:
matrix_origin_1 = np.array([[1, 1, 1, 0, 1],
                   [1, 0, 1, 1, 0],
                   [0, 1, 0, 1, 1],
                   [1, 1, 0, 1, 0]])

In [ ]:
matrix_origin_2 = np.array([[0, 0, 0, 0],
                            [1, 1, 1, 0],
                            [0, 1, 1, 1],
                            [1, 0, 0, 1]])

In [ ]:
def ref(matrix):
  notZero = []
  for j in range(matrix.shape[1]):
    if 1 in matrix[:, j]:
      notZero = np.where(matrix[:, j] == 1)
      matrix[[0,notZero[0][0]]] = matrix[[notZero[0][0], 0]]
      for k in range(1, notZero[0].size):
        matrix[notZero[0][k], :] += matrix[0, :]
        matrix[notZero[0][k], :] = matrix[notZero[0][k], :] % 2
      break

  if matrix.shape[0] == 1:
    return
  ref(matrix[1:, :])
  return

matrix_triangle = matrix_origin_1.copy()
ref(matrix_triangle)
print(matrix_triangle)

[[1 1 1 0 1]
 [0 1 0 1 1]
 [0 0 1 1 1]
 [0 0 0 0 0]]


1.2. Реализовать функцию RREF(), приводящую матрицу к
приведённому ступенчатому виду.

In [ ]:
def rref(matrix):
  ref(matrix)
  j_lead = 0
  for i in range(1, matrix.shape[0]):
    notZero = np.where(matrix[i, :] == 1)
    if notZero[0].size ==0 :
      return matrix
    j_lead = notZero[0][0]
    for k in range(0,i):
      if matrix[k,j_lead]:
        matrix[k,:] += matrix[i,:]
        matrix[k,:] = matrix[k,:]%2
  return matrix


matrix_triangle_rref = matrix_origin_1.copy()
rref(matrix_triangle_rref)
print(matrix_triangle_rref)

[[1 0 0 0 1]
 [0 1 0 1 1]
 [0 0 1 1 1]
 [0 0 0 0 0]]


1.3.1 На основе входной матрицы сформировать порождающую
матрицу в ступенчатом виде.

In [ ]:
class LinearCode:  # класс линейных кодов

  def __init__(self, matrix):
    self.matrix = matrix
    self.lead = []
    self.H = []

  def ref(self,matrix):
    notZero = []
    for j in range(matrix.shape[1]):
      if 1 in matrix[:, j]:
        notZero = np.where(matrix[:, j] == 1)
        matrix[[0,notZero[0][0]]] = matrix[[notZero[0][0], 0]]
        for k in range(1, notZero[0].size):
          matrix[notZero[0][k], :] += matrix[0, :]
          matrix[notZero[0][k], :] = matrix[notZero[0][k], :] % 2
        break
    if matrix.shape[0] == 1:
      return
    self.ref(matrix[1:, :])
    return

  def rref(self):
    self.ref(self.matrix)
    j_lead = 0
    for i in range(1, self.matrix.shape[0]):
      notZero = np.where(self.matrix[i, :] == 1)
      if notZero[0].size ==0 :
        return self.matrix
      j_lead = notZero[0][0]
      for k in range(0,i):
        if self.matrix[k,j_lead]:
          self.matrix[k,:] += self.matrix[i,:]
          self.matrix[k,:] = self.matrix[k,:]%2
    return self.matrix

  def delete_null_rows(self):  # удаляем нулевые строки
    for i in range(0,self.matrix.shape[0]):
      if 1 not in self.matrix[i,:]:
        self.matrix = np.delete(self.matrix,i,axis=0)
    return self.matrix

  def count_k_n(self):  # задаем  n - число столбцов, k - число строк
    return(self.matrix.shape[1],self.matrix.shape[0])

  def search_lead(self):  # фиксируем ведубщие столбцы
    lead = []
    for i in range(0, self.matrix.shape[0]):
      notZero = np.where(self.matrix[i, :] == 1)
      if notZero[0].size ==0 :
        self.lead = lead
        return lead
      else:
        lead.append(notZero[0][0])
    self.lead = lead
    return lead

  def delete_lead_colums(self):  # удаляем ведющие столбцы матрицы
    lead_colums = self.lead
    for i in range(0,len(lead_colums)):
      self.matrix = np.delete(self.matrix,lead_colums[i]-i,axis = 1)
    return self.matrix


  def checking_matrix(self):  # создаем проверочную матрицу H
    check_matrix = []
    temp = np.eye(self.matrix.shape[1],dtype = int)
    k = 0
    m = 0
    for i in range(0, self.matrix.shape[0] + self.matrix.shape[1]):
      if i in self.lead:
        check_matrix.append(self.matrix[k])
        k += 1
      else:
        check_matrix.append(temp[m])
        m +=1
    check_matrix = np.array(check_matrix)
    self.H = check_matrix
    return check_matrix


  def generate_code_words_1(self):  # формируем кодовые слова, складывая все слова из порождающего множества
    zer = np.zeros(self.matrix.shape[1], dtype=int)
    final_word = set()
    final_word.add(tuple(zer.tolist()))
    for i in range(1, self.matrix.shape[0] + 1):
      comd_word = list(itertools.combinations(range(self.matrix.shape[0]), i))
      for comb in comd_word:
        word = np.zeros(self.matrix.shape[1], dtype=int)
        for j in comb:
          word += self.matrix[j, :]
        word %= 2
        final_word.add(tuple(word.tolist()))
    return np.array(list(final_word))

  def generate_code_words_2(self):  # формируем кодовые слова, беря все кодовые слова длины k и умножая на матрицу
    n,k = self.count_k_n()
    final_code = itertools.product(range(2), repeat=k)
    answer = []
    for code in (final_code):
      one_word = np.dot(code,self.matrix)%2
      answer.append(one_word)
    return answer





In [ ]:
# пример по созданию проверочной матрицы на основе порождающей
matrix_example_1 = np.array([[1, 1, 1, 0, 1],
              [1, 0, 1, 1, 0],
              [0, 1, 0, 1, 1],
              [1, 1, 0, 1, 0]])

cl = LinearCode(matrix_example_1)

num_one = cl.rref()
print(num_one,'\n')

num_one = cl.delete_null_rows()
print(num_one,'\n')

n,k = cl.count_k_n()
print(n,k,'\n')

lead = cl.search_lead()
print(lead,'\n')

new_matrix = cl.delete_lead_colums()
print(new_matrix,'\n')

final_matrix = cl.checking_matrix()
print(np.array(final_matrix),'\n')

[[1 0 0 0 1]
 [0 1 0 1 1]
 [0 0 1 1 1]
 [0 0 0 0 0]] 

[[1 0 0 0 1]
 [0 1 0 1 1]
 [0 0 1 1 1]] 

5 3 

[0, 1, 2] 

[[0 1]
 [1 1]
 [1 1]] 

[[0 1]
 [1 1]
 [1 1]
 [1 0]
 [0 1]] 



In [ ]:
# 2 пример
matrix_example_2 = np.array(
    [[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
     [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
     [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
     [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]])

cl_2 = LinearCode(matrix_example_2)

num_two = cl_2.rref()
num_two = cl_2.delete_null_rows()
n_2,k_2 = cl_2.count_k_n()
print(n_2,k_2)
lead_2 = cl_2.search_lead()
new_matrix_2 = cl_2.delete_lead_colums()

final_matrix_2 = cl_2.checking_matrix()
print(final_matrix_2,'\n')


11 5
[[0 1 1 1 1 0]
 [1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 1 1]
 [0 0 0 1 0 1]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 1]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]] 



1.4. Сформировать все кодовые слова длины n двумя способами.

1.4.1 Сложить все слова из порождающего множества, оставить
неповторяющиеся.

1.4.2 Взять все двоичные слова длины k, умножить каждое на G

In [ ]:
matrix_G = np.array(
    [[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
     [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
     [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
     [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]])

matrix_H = np.array(
    [[0, 1, 1, 1, 1, 0],
     [1, 0, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0],
     [0, 0, 1, 0, 1, 1],
     [0, 0, 0, 1, 0, 1],
     [0, 0, 1, 0, 0, 0],
     [0, 0, 0, 0, 1, 0],
     [0, 0, 0, 1, 0, 0],
     [0, 0, 0, 0, 1, 1],
     [0, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 1]] )

cl_3 = LinearCode(matrix_G)

code_word_1 = np.array(cl_3.generate_code_words_1()) #1 способ формирования кодовых слов
print(code_word_1,'\n')

set_code_word_1 = set()
for word in (code_word_1):
  set_code_word_1.add(tuple(word.tolist()))


code_word_2 = np.array(cl_3.generate_code_words_2())#2 способ формирования кодовых слов
print(code_word_2,'\n')

set_code_word_2 = set()
for word in (code_word_2):
  set_code_word_2.add(tuple(word.tolist()))

print(set_code_word_1 == set_code_word_2) #проверяем, равны ли эти множества кодовых


check_zero_matr = []  # проверка на умножение
for code in (code_word_2):
  one_word = np.dot(code,matrix_H)%2
  check_zero_matr.append(one_word)
print(np.array(check_zero_matr))

[[0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 1 1 1 1 1 1 1 1]
 [1 0 1 0 1 1 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 1 0 1]
 [1 0 1 1 1 0 1 0 0 1 0]
 [1 0 1 1 0 0 1 1 1 0 0]
 [0 0 0 0 1 0 0 1 1 1 0]
 [0 0 0 1 1 1 1 1 0 0 0]
 [0 0 0 1 1 1 0 1 0 1 0]
 [0 0 0 0 1 0 1 1 1 0 0]
 [1 0 1 1 0 0 1 1 0 1 1]
 [0 0 0 1 1 1 0 1 1 0 1]
 [1 0 1 0 0 1 1 1 0 0 0]
 [1 0 1 0 0 1 1 1 1 1 1]
 [0 0 0 1 0 1 0 0 1 0 0]
 [0 0 0 0 1 0 1 1 0 1 1]
 [0 0 0 0 1 0 0 1 0 0 1]
 [1 0 1 0 1 1 0 0 0 1 1]
 [1 0 1 0 0 1 0 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 1 1 0 0 0 1 1 1]
 [1 0 1 1 1 0 0 0 0 0 0]
 [1 0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 1 1 0 0 0 1]
 [1 0 1 0 1 1 1 0 1 1 0]
 [1 0 1 1 0 0 0 1 1 1 0]
 [1 0 1 0 1 1 1 0 0 0 1]
 [0 0 0 1 0 1 1 0 1 1 0]
 [1 0 1 0 0 1 0 1 1 0 1]
 [0 0 0 0 0 0 0 0 1 1 1]
 [1 0 1 1 0 0 0 1 0 0 1]
 [0 0 0 1 0 1 0 0 0 1 1]] 

[[0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 1 0 1 0 1]
 [0 0 0 0 1 0 0 1 0 0 1]
 [0 0 0 0 1 0 0 1 1 1 0]
 [0 0 0 0 1 0 1 1 0 1 1]
 [0 0 0 0 1 0 1 1 1 0 

1.4 Вычислить кодовое расстояние получившегося кода.

Сделать вывод о кратности обнаруживаемой ошибки t.

In [ ]:
sums = np.sum(matrix_G, axis=1)
d = sums[np.argmin(sums)]
t = d - 1

print("Result:")
print("d = ", d)
print("t = ", t)

Result:
d =  2
t =  1


1.4.1 Внести в кодовое слово ошибку кратности не более t, умножить
полученное слово на H, убедиться в обнаружении ошибки.

In [ ]:
v = np.array([1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0])
e1 = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
s = (v + e1)%2
print('v + e1 = ',s)

s2 = np.dot(s,matrix_H)%2
print('(v + e1)@H = ',s2,' - error')

v + e1 =  [1 0 0 1 1 0 1 0 0 1 0]
(v + e1)@H =  [0 1 0 0 0 0]  - error


1.4.2 Найти для некоторого кодового слова ошибку кратности t+1
такую, что при умножении на H ошибка не может быть обнаружена.

In [ ]:
is_find = 0
print(v)
for i in range(len(v)):
  if is_find:
    break
  for j in range(len(v)):
    if i != j:
      e2 = np.zeros(len(v), dtype=int)
      e2[i], e2[j] = 1, 1

      s = (v + e2) % 2
      s2 = np.dot(s, matrix_H)%2
      if sum(s2) == 0:
        is_find = 1
        break

print('e2', e2)
print('v + e2 = ', s)
print('(v + e2)@H = ', s2,' - no error')

[1 0 1 1 1 0 1 0 0 1 0]
e2 [0 0 0 0 0 0 1 0 0 1 0]
v + e2 =  [1 0 1 1 1 0 0 0 0 0 0]
(v + e2)@H =  [0 0 0 0 0 0]  - no error
